# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
type(cur)

psycopg2.extensions.cursor

In [11]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [12]:
song_files = get_files('data/song_data')
type(song_files)

list

In [13]:
filepath = song_files[0]

In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:
song_df = df.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']]
song_df

,song_id,title,artist_id,year,duration
0,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,ARNF6401187FB57032,1994,305.162


In [18]:
song_data = tuple(song_df.values[0])
song_data

('SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
artist_df = df.loc[:, ['artist_id', 'artist_name', 
                          'artist_location', 'artist_latitude', 'artist_longitude']]
artist_df

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644


In [20]:
artist_data = tuple(artist_df.values[0])
artist_data

('ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data1)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data/2018/11")

In [13]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-09-events.json'

In [14]:
df = pd.read_json(filepath, lines=True)
print(df.shape)
df.head(3)

(283, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds

In [15]:
next_song_filter = df['page'] == 'NextSong'
df = df[next_song_filter]
print(df.shape)
df.head(3)

(252, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


Verify that the dataframe only contains `NextSong` pages.

In [16]:
df['page'].value_counts()

NextSong    252
Name: page, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 282
Data columns (total 18 columns):
artist           252 non-null object
auth             252 non-null object
firstName        252 non-null object
gender           252 non-null object
itemInSession    252 non-null int64
lastName         252 non-null object
length           252 non-null float64
level            252 non-null object
location         252 non-null object
method           252 non-null object
page             252 non-null object
registration     252 non-null float64
sessionId        252 non-null int64
song             252 non-null object
status           252 non-null int64
ts               252 non-null int64
userAgent        252 non-null object
userId           252 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 37.4+ KB


Convert the timestamp `ts` to a `datetime` object.

In [18]:
df['ts'] = pd.to_datetime(df['ts'], unit = 'ms')
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,2018-11-09 00:06:17.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,2018-11-09 00:09:46.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,2018-11-09 00:12:27.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`

* Test the `dt` attribute calls. Then create the columns and retrieve their corresponding values.
* Convert the np.datetime object to string so it can be loaded into table.

In [19]:
list(np.datetime_as_string(df['ts'].values[:3]))

['2018-11-09T00:06:17.796000000',
 '2018-11-09T00:09:46.796000000',
 '2018-11-09T00:12:27.796000000']

In [20]:
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']
time_data = [list(np.datetime_as_string(df['ts'].values)), list(df['ts'].dt.hour.values), 
             list(df['ts'].dt.day.values), list(df['ts'].dt.week.values), 
             list(df['ts'].dt.month.values), list(df['ts'].dt.year.values), list(df['ts'].dt.weekday.values)]

- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
time_dict = dict(zip(column_labels, time_data))
time_dict.keys()

dict_keys(['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

In [22]:
time_df = pd.DataFrame(time_dict)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-09T00:06:17.796000000,0,9,45,11,2018,4
1,2018-11-09T00:09:46.796000000,0,9,45,11,2018,4
2,2018-11-09T00:12:27.796000000,0,9,45,11,2018,4
3,2018-11-09T00:14:52.796000000,0,9,45,11,2018,4
4,2018-11-09T00:17:44.796000000,0,9,45,11,2018,4


In [23]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
timestamp    252 non-null object
hour         252 non-null int64
day          252 non-null int64
week         252 non-null int64
month        252 non-null int64
year         252 non-null int64
weekday      252 non-null int64
dtypes: int64(6), object(1)
memory usage: 13.9+ KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [26]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
1,42,Harper,Barrett,M,paid
2,42,Harper,Barrett,M,paid
3,42,Harper,Barrett,M,paid
4,42,Harper,Barrett,M,paid


Duplicate values exist but this table should have unique users based on their ID.

In [27]:
user_df = user_df.drop_duplicates(subset='userId')
user_df.head()

,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
34,49,Chloe,Cuevas,F,free
35,6,Cecilia,Owens,F,free
36,24,Layla,Griffin,F,paid
38,80,Tegan,Levine,F,paid


Verify that `userID` values are unique.

In [28]:
pd.Series(user_df['userId']).is_unique

True

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [29]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

Create the query.

In [32]:
q2 = """
SELECT
    s.id song_id,
    a.id artist_id
FROM songs s
LEFT JOIN artists a
    ON s.artist_id = a.id
WHERE s.title = (%s)
AND a.name = (%s)
AND s.duration = (%s);"""

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.
- Create the `songplay_data` table


In [33]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [34]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit = 'ms'), row.userId, row.level,
                     songid, artistid, row.sessionId,
                     row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [35]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.